<a href="https://colab.research.google.com/github/veager/StudyNotes/blob/new/Codes/PyTorch-Tutorial/PyTorch-%E6%8D%9F%E5%A4%B1%E5%87%BD%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch 损失函数 Loss Functions

参考资料：

- 博客：PyTorch 激活函数，[地址](https://www.cnblogs.com/veager/articles/16314647.html)

- Github: Codes/PyTorch-Tutorial/PyTorch-损失函数.ipynb

# 0 概述

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 0.1 主要参数

## 0.2 批损失函数与总损失函数

In [2]:
# 定义 损失函数 MSE 损失
def loss_func(model_out, target, reduction='mean'):
    loss = F.mse_loss(model_out, target, reduction=reduction) 
    return loss

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

torch.manual_seed(0)
# 随机生成 100 对数据
n_sample = 100
y_pred = torch.rand(n_sample)
y_true = torch.rand(n_sample)
print('MSE', F.mse_loss(y_pred, y_true).item())

dl = DataLoader(TensorDataset(y_pred, y_true), batch_size=17)

MSE 0.14062897861003876


In [4]:
total_loss = 0.
for i, (y_pred_batch, y_true_batch) in enumerate(dl):                     
    batch_loss = loss_func(y_pred_batch, y_true_batch, reduction='mean')  # 返回的 loss 为 一个 batch 的平均损失
    total_loss += batch_loss.item()  # total lost 为所有 batch 的平均损失的和，而不是整个训练集的平均损失
print('Sum of batch loss:', total_loss)

total_loss = 0.
for i, (y_pred_batch, y_true_batch) in enumerate(dl):                       
    batch_loss = loss_func(y_pred_batch, y_true_batch, reduction='sum')  # 返回的 loss 为 一个 batch 的总损失
    total_loss += batch_loss.item()  # total lost 为所有 batch 的平均损失的和
total_loss = total_loss / n_sample
print('MSE:', total_loss)

Sum of batch loss: 0.8410508707165718
MSE: 0.1406289792060852


# 1 回归损失

- `reduction` 参数

- `nn.MSELoss()`

- `nn.L1Loss()`

In [5]:
input = torch.randn(3, 5)
target = torch.randn(3, 5)

# 损失函数 1：reduction='mean'
output_1 = nn.MSELoss()(input, target)
print(output_1.size(), output_1)
# Output 1：torch.Size([]) tensor(1.1773)

# 损失函数 2：reduction='none'
output_2 = nn.MSELoss(reduction='none')(input, target)
print(output_2.size(), output_2)
# Output 2：torch.Size([3, 5]) tensor([[...

torch.Size([]) tensor(2.6747)
torch.Size([3, 5]) tensor([[3.2589e-01, 1.9221e+00, 2.8297e+00, 2.4167e+00, 1.6791e+01],
        [3.9843e+00, 7.2390e-02, 6.3163e+00, 9.4038e-01, 3.1233e-01],
        [3.4942e+00, 2.6550e-01, 6.4022e-02, 6.7654e-03, 3.7880e-01]])


# 2 二分类损失

- `nn.BCEWithLogitsLoss()` 层 和 `F.binary_cross_entropy_with_logits()` 函数

- `nn.BCELoss()` 层 和 `F.binary_cross_entropy()` 函数

In [6]:
torch.manual_seed(0)  # 固定随机数种子
input = torch.rand(5)  # 生成5个 [0,1) 之间的随机数
target = torch.empty(5).random_(2)
print(input, target)

# 方式一：使用层类：nn.BCEWithLogitsLoss() 
loss_layer1 = nn.BCEWithLogitsLoss()
output_1 = loss_layer1(input, target)
print(output_1)

# 方式二：使用层类：nn.BCELoss() + nn.Sigmoid()
loss_layer2 = nn.BCELoss()
sigmoid_layer = nn.Sigmoid()
input_2 = sigmoid_layer(input)
output_2 = loss_layer2(input_2, target)
print(output_2)

# 方式三：使用函数：F.binary_cross_entropy_with_logits() 
output_3 = F.binary_cross_entropy_with_logits(input, target)
print(output_3)

# 方式四：使用函数：torch.sigmoid() + F.binary_cross_entropy() 
input_4 = torch.sigmoid(input)
output_4 = F.binary_cross_entropy(input_4, target)
print(output_4)

tensor([0.4963, 0.7682, 0.0885, 0.1320, 0.3074]) tensor([1., 1., 1., 1., 1.])
tensor(0.5374)
tensor(0.5374)
tensor(0.5374)
tensor(0.5374)


# 3 多分类损失

- `nn.CrossEntropyLoss()` 层和函数 `F.cross_entropy()` 函数

- `nn.NLLLoss()` 层和函数 `F.nll_loss()` 函数

In [7]:
input = torch.randn(7, 3)
target = torch.empty(7, dtype=torch.long).random_(3)
print(input)
print(target)

# 方式 1：nn.CrossEntropyLoss()
output_1 = nn.CrossEntropyLoss()(input, target)

# 方式 2：nn.LogSoftmax() + nn.NLLLoss() 
input_2 = nn.LogSoftmax(dim=1)(input)
output_2 = nn.NLLLoss()(input_2, target)

# 方式 3：cross_entropy()
output_3 = F.cross_entropy(input, target)

# 方式 4：torch.log_softmax() + F.nll_loss()
input_4 = torch.log_softmax(input, dim=1)
output_4 = F.nll_loss(input_4, target)

tensor([[ 0.4913, -0.2041, -0.0885],
        [ 0.5239, -0.6659, -0.1116],
        [-0.6136,  0.0316, -0.4927],
        [ 0.2484,  0.4397,  0.1124],
        [ 0.6408, -2.3160, -0.1023],
        [ 0.7924, -0.2897,  0.0525],
        [ 0.5229,  2.3022, -1.4689]])
tensor([1, 2, 1, 1, 2, 2, 1])
